## Check the test dynamic_offset.txt files for self-consistency

The expectation is that `calc_aca_from_targ(target, y_off, z_off, SI_ALIGN)` should be the same as the `aca` coordinate.  Here `y_off` is the sum of the two offsets `target_offset_y + aca_offset_y`.

Instead there is a residual which looks like the CHARACTERISTICS offset (around -3, +11 arcsec).

A critical routine in this process is [calc_aca_from_targ](http://cxc.cfa.harvard.edu/mta/ASPECT/tool_doc/chandra_aca/_modules/chandra_aca/transform.html#calc_aca_from_targ), which is validated in the second section of this notebook.

In [1]:
from __future__ import division, print_function
import glob
import shelve
from astropy.table import Table
import numpy as np
from Quaternion import Quat
import chandra_aca
from chandra_aca import calc_aca_from_targ, calc_targ_from_aca
import parse_cm
from Ska.engarchive import fetch_sci
from Chandra.Time import DateTime
import Ska.Shell
import functools
import mica.archive.obspar
import Ska.arc5gl
import os
import parse_cm.maneuver

%matplotlib inline

In [2]:
# SI_ALIGN from Matlab code
SI_ALIGN = chandra_aca.ODB_SI_ALIGN
SI_ALIGN

array([[  9.99999906e-01,  -3.37419984e-04,  -2.73439987e-04],
       [  3.37419984e-04,   9.99999943e-01,  -4.61320600e-08],
       [  2.73439987e-04,  -4.61320600e-08,   9.99999963e-01]])

In [3]:
def print_dq(q1, q2):
    """
    Print the difference between two quaternions
    """
    dq = q1.inv() * q2
    dr, dp, dy, _ = np.degrees(dq.q) * 2 * 3600
    print('droll={:6.2f}, dpitch={:6.2f}, dyaw={:6.2f} arcsec'.format(dr, dp, dy))

In [4]:
def check_obs(obs):
    """
    Check `obs` (which is a row out of the dynamic offsets table) for consistency
    between target and aca coordinates given the target and aca offsets and the
    SI_ALIGN alignment matrix
    """
    y_off = (obs['target_offset_y'] + obs['aca_offset_y']) / 3600
    z_off = (obs['target_offset_z'] + obs['aca_offset_z']) / 3600
    
    q_targ = Quat([obs['target_ra'], obs['target_dec'], obs['target_roll']])
    q_aca = Quat([obs['aca_ra'], obs['aca_dec'], obs['aca_roll']])
    
    q_aca_out = calc_aca_from_targ(q_targ, y_off, z_off, SI_ALIGN)
    print('Obsid={} detector={:6s} '.format(obs['obsid'], obs['detector']), end='')
    print_dq(q_aca, q_aca_out)

In [5]:
def check_obs_vs_manvr(obs, manvr):
    """
    Check against attitude from actual flight products (from maneuver summary file)
    """
    mf = manvr['final']
    q_flight = Quat([mf['q1'], mf['q2'], mf['q3'], mf['q4']])
    q_aca = Quat([obs['aca_ra'], obs['aca_dec'], obs['aca_roll']])
    print('Obsid={} detector={:6s} chipx={:8.2f} chipy={:8.2f} '
          .format(obs['obsid'], obs['detector'], obs['chipx'], obs['chipy']), end='')
    print_dq(q_aca, q_flight)

In [6]:
dat = Table.read('MAR0516O/MAR0516O_aimpoint_adjustment.txt', format='ascii')

In [7]:
dat[10:30]

obsid,detector,chipx,chipy,chip_id,target_offset_y,target_offset_z,target_ra,target_dec,target_roll,aca_offset_y,aca_offset_z,aca_ra,aca_dec,aca_roll,mean_date,mean_t_ccd
int64,string48,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string168,float64
18272,ACIS-I,930.2,1009.6,3,-18.0,-18.0,194.649583,28.074806,135.000219,13.31,3.48,194.672889,28.070562,134.989033,2016:068:03:12:27.816,-15.48
18790,ACIS-S,210.0,520.0,7,0.0,-18.0,111.463705,-7.735581,280.000024,9.91,-16.7,111.454022,-7.715341,279.9987,2016:068:10:43:27.816,-15.39
18791,ACIS-I,930.2,1009.6,3,-18.0,-18.0,194.649583,28.074806,135.000219,11.03,2.37,194.672136,28.07079,134.989387,2016:068:20:14:43.816,-14.9
18800,ACIS-S,210.0,520.0,7,0.0,-18.0,111.463705,-7.735581,280.000024,7.84,-17.71,111.454402,-7.715858,279.998751,2016:069:05:45:59.816,-14.87
18792,ACIS-I,930.2,1009.6,3,-18.0,-18.0,194.649583,28.074806,135.000219,9.34,1.55,194.671575,28.07096,134.989651,2016:069:13:25:11.816,-14.47
18469,ACIS-I,930.2,1009.6,3,-60.0,-18.0,202.907917,-17.611944,58.755253,13.49,3.59,202.915071,-17.623011,58.757361,2016:070:07:52:11.816,-15.51
18194,ACIS-S,200.7,476.9,7,0.0,-18.0,209.010833,18.371611,103.000066,15.25,4.87,209.028546,18.351803,102.994421,2016:070:11:11:43.816,-15.58
18801,ACIS-S,205.0,480.0,7,0.0,-18.0,257.274167,23.224389,88.000033,11.97,2.77,257.28566,23.201836,87.995471,2016:070:17:09:47.816,-15.3
18793,ACIS-I,930.2,1009.6,3,-18.0,-18.0,194.649583,28.074806,135.000219,8.79,1.27,194.671389,28.071015,134.989738,2016:071:06:11:31.816,-14.33


In [8]:
# Check each observation
for obs in dat:
    check_obs(obs)

Obsid=18168 detector=ACIS-S droll= -1.32, dpitch=  0.33, dyaw= -1.76 arcsec
Obsid=18091 detector=ACIS-S droll= -1.60, dpitch=  0.34, dyaw= -1.76 arcsec
Obsid=18157 detector=ACIS-S droll= -0.18, dpitch=  0.34, dyaw= -1.75 arcsec
Obsid=18725 detector=ACIS-S droll=  0.86, dpitch=  0.34, dyaw= -1.76 arcsec
Obsid=18276 detector=ACIS-I droll=  0.69, dpitch=  0.34, dyaw= -1.76 arcsec
Obsid=18201 detector=ACIS-S droll=  0.80, dpitch=  0.33, dyaw= -1.75 arcsec
Obsid=18803 detector=ACIS-S droll=  0.04, dpitch=  0.33, dyaw= -1.75 arcsec
Obsid=17720 detector=ACIS-S droll=  0.02, dpitch=  0.34, dyaw= -1.75 arcsec
Obsid=18304 detector=HRC-I  droll= -0.00, dpitch= -0.01, dyaw=  0.00 arcsec
Obsid=18212 detector=ACIS-S droll=  1.04, dpitch=  0.34, dyaw= -1.75 arcsec
Obsid=18272 detector=ACIS-I droll=  0.79, dpitch=  0.33, dyaw= -1.76 arcsec
Obsid=18790 detector=ACIS-S droll=  0.09, dpitch=  0.34, dyaw= -1.75 arcsec
Obsid=18791 detector=ACIS-I droll=  0.79, dpitch=  0.33, dyaw= -1.75 arcsec
Obsid=18800 

In [9]:
mm = parse_cm.maneuver.read_maneuver_summary('MAR0516O/mm065_0409.sum', structured=True)
mm = {m['final']['id']: m for m in mm}  # Turn into a dict

In [10]:
for obs in dat:
    check_obs_vs_manvr(obs, mm[obs['obsid'] * 100])

Obsid=18168 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll= -0.00, dpitch=  0.76, dyaw= -3.34 arcsec
Obsid=18091 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll= -0.01, dpitch=  1.21, dyaw= -4.25 arcsec
Obsid=18157 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll=  0.01, dpitch=  1.79, dyaw= -5.43 arcsec
Obsid=18725 detector=ACIS-S chipx=  210.00 chipy=  520.00 droll= -0.01, dpitch= 23.40, dyaw=-10.85 arcsec
Obsid=18276 detector=ACIS-I chipx=  930.20 chipy= 1009.60 droll= -0.00, dpitch=  0.64, dyaw= -1.10 arcsec
Obsid=18201 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll= -0.01, dpitch= -1.21, dyaw=  0.70 arcsec
Obsid=18803 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll= -0.00, dpitch= -1.32, dyaw=  0.94 arcsec
Obsid=17720 detector=ACIS-S chipx=  200.70 chipy=  476.90 droll=  0.00, dpitch= -2.23, dyaw=  2.80 arcsec
Obsid=18304 detector=HRC-I  chipx= 7591.00 chipy= 7936.10 droll=  0.01, dpitch= -2.41, dyaw= 17.61 arcsec
Obsid=18212 detector=ACIS-S chipx=  200.70 chi

### Use dmcoords to relate detector, dy, dz to chipx, chipy

This will be an approximation that applies over small displacements (of order 100 pixels).

In [15]:
ciaoenv = Ska.Shell.getenv('source /soft/ciao/bin/ciao.sh')
ciaorun = functools.partial(Ska.Shell.bash, env=ciaoenv)

In [14]:
dmcoords_cmd = ['dmcoords', 'none',
                'asolfile=none',
                'detector="{detector}"',
                'fpsys="{fpsys}"',
                'opt=cel',
                'ra={ra_targ}', 
                'dec={dec_targ}',
                'celfmt=deg', 
                'ra_nom={ra_nom}',
                'dec_nom={dec_nom}',
                'roll_nom={roll_nom}',  
                'ra_asp=")ra_nom"',
                'dec_asp=")dec_nom"',
                'roll_asp=")roll_nom"',      
                'sim="{sim_x} 0 {sim_z}"',
                'displace="0 {dy} {dz} 0 0 0"',       
                'verbose=0']
dmcoords_cmd = ' '.join(dmcoords_cmd)

In [16]:
def dmcoords_chipx_chipy(keys, verbose=False):
    """
    Get the dmcoords-computed chipx and chipy for given detector and
    aspect solution DY and DZ values.  NOTE: the ``dy`` and ``dz`` inputs
    to dmcoords are flipped in sign from the ASOL values.  Generally the
    ASOL DY/DZ are positive and dmcoord input values are negative.  This
    sign flip is handled *here*, so input to this is ASOL DY/DZ.
    
    :param keys: dict of event file keywords
    """
    # See the absolute_pointing_uncertainty notebook in this repo for the
    # detailed derivation of this -15.5, 6.0 arcsec offset factor.  See the
    # cell below for the summary version.
    ciaorun('punlearn dmcoords')
    fpsys_map = {'HRC-I': 'HI1',
                'HRC-S': 'HS2',
                'ACIS': 'ACIS'}
    keys = {key.lower(): val for key, val in keys.items()}
    det = keys['detnam']
    keys['detector'] = (det if det.startswith('HRC') else 'ACIS')
    keys['dy'] = -keys['dy_avg']
    keys['dz'] = -keys['dz_avg']
    keys['fpsys'] = fpsys_map[keys['detector']]
    
    cmd = dmcoords_cmd.format(**keys)
    ciaorun(cmd)
    
    if verbose:
        print(cmd)
    return [float(x) for x in ciaorun('pget dmcoords chipx chipy chip_id')]

In [17]:
def get_evt_meta(obsid, detector):
    evts = shelve.open('event_meta.shelf')
    sobsid = str(obsid)
    if sobsid not in evts:
        det = 'hrc' if detector.startswith('HRC') else 'acis'
        arc5gl = Ska.arc5gl.Arc5gl()
        arc5gl.sendline('obsid={}'.format(obsid))
        arc5gl.sendline('get {}2'.format(det) + '{evt2}')
        del arc5gl

        files = glob.glob('{}f{}*_evt2.fits.gz'.format(det, obsid))
        if len(files) != 1:
            raise ValueError('Wrong number of files {}'.format(files))
        evt2 = Table.read(files[0])
        os.unlink(files[0])

        evts[sobsid] = {k.lower(): v for k, v in evt2.meta.items()}

    out = evts[sobsid]
    evts.close()
    
    return out

In [18]:
def check_predicted_chipxy(obs):
    obsid = obs['obsid']
    detector = obs['detector']
    try:
        evt = get_evt_meta(obsid, detector)
    except ValueError as err:
        print('Obsid={} detector={}: fail {}'.format(obsid, detector, err))
        return
    f_chipx, f_chipy, f_chip_id = dmcoords_chipx_chipy(evt)
    
    q_nom_flight = Quat([evt['ra_nom'], evt['dec_nom'], evt['roll_nom']])
    q_aca = Quat([obs['aca_ra'], obs['aca_dec'], obs['aca_roll']])
    mf = mm[obsid * 100]['final']
    q_flight = Quat([mf['q1'], mf['q2'], mf['q3'], mf['q4']])
    dq = q_flight.dq(q_aca)
    q_nom_test = q_nom_flight * dq
    evt_test = dict(evt)
    evt_test['ra_nom'] = q_nom_test.ra
    evt_test['dec_nom'] = q_nom_test.dec
    evt_test['roll_nom'] = q_nom_test.roll
    
    scale = 0.13175 if detector.startswith('HRC') else 0.492
    aim_chipx = obs['chipx']
    aim_chipy = obs['chipy']
    if detector == 'ACIS-S':  # SIGNS?
        aim_chipx += -obs['target_offset_y'] / scale  # SIGN??
        aim_chipy += -obs['target_offset_z'] / scale  # 
    elif detector == 'ACIS-I':
        aim_chipx += -obs['target_offset_z'] / scale
        aim_chipy += +obs['target_offset_y'] / scale

    chipx, chipy, chip_id = dmcoords_chipx_chipy(evt_test)
    print('{} {:6s} sim_z={:6.1f} aimpoint:{:6.1f},{:6.1f} test:{:6.1f},{:6.1f} '
          'flight:{:6.1f},{:6.1f} delta: {:.1f} arcsec'
         .format(obsid, detector, evt['sim_z'], aim_chipx, aim_chipy, chipx, chipy, f_chipx, f_chipy,
                np.hypot(aim_chipx - chipx, aim_chipy - chipy) * scale))

In [19]:
for obs in dat:
    check_predicted_chipxy(obs)

18168 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 174.2, 499.7 flight: 167.4, 498.2 delta: 14.7 arcsec
18091 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 179.3, 499.4 flight: 170.7, 497.0 delta: 12.6 arcsec
18157 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 185.5, 500.9 flight: 174.4, 497.2 delta: 9.7 arcsec
18725 ACIS-S sim_z=-190.1 aimpoint: 210.0, 556.6 test: 199.9, 548.9 flight: 177.8, 501.3 delta: 6.2 arcsec
18276 ACIS-I sim_z=-233.6 aimpoint: 966.8, 973.0 test: 964.6, 977.7 flight: 963.3, 979.9 delta: 2.5 arcsec
18201 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 196.8, 512.8 flight: 198.3, 515.3 delta: 1.9 arcsec
18803 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 197.8, 513.1 flight: 199.7, 515.8 delta: 1.4 arcsec
17720 ACIS-S sim_z=-190.1 aimpoint: 200.7, 513.5 test: 197.6, 511.4 flight: 203.2, 515.9 delta: 1.9 arcsec
18304 HRC-I  sim_z= 127.0 aimpoint:7591.0,7936.1 test:7566.9,7956.4 flight:7648.5,7848.9 delta: 4.2 arcsec
18212 ACIS-S sim_z=-190.1 aimpoint: